In [56]:
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os

In [57]:
# Read in PSD to dataframe(s)
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\git\grundfos-express-tools\psd to xml\PSD files for testing"
file = "Lbom-ES.xlsx"
psd_file = os.path.join(myDir, file)
# psd_file = "VLC-VLSCbom.xlsx"
tabName = "Impeller"

In [58]:
# Creates dataframe from excel tab of interest
raw_data = pd.read_excel(psd_file, sheet_name=tabName, header=1, index_col=False)
unnamed_cols = raw_data.columns.str.match("Unnamed")
psd_data = raw_data.drop(raw_data[raw_data.columns[unnamed_cols]], axis=1) # Drops columns with unnamed cols. Need to ask if these columns should be preserved.

In [59]:
def get_row_index(search_query:str, df):
    """Returns row integer number of search_query from 1st column"""
    row_num = df.index[df.iloc[:,0]==search_query].tolist()[0]
    return row_num
    
# Finds [START], [END] row # to separate header from data rows
start_row = get_row_index("[START]", psd_data)
end_row = get_row_index("[END]", psd_data)

In [60]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'UTF-8')
    reparsed = minidom.parseString(rough_string)
    print(reparsed.toprettyxml(indent="  "))

In [61]:
# Create dict of column headers with data types for each column. Skips first column from PSD
header_dict = {i: psd_data.at[1, i] for i in psd_data.columns if psd_data.columns.get_loc(i) != 0}

# PSD field says "pointer-merge", but this turns into "text" during XML conversion process anyway.
header_dict.update({'ID': "text"}) 
print(header_dict)

{'ID': 'text', 'Model': 'text', 'CodeX': 'text', 'ImpellerMaterial': 'text', 'PacoMatlCode': 'text', 'CapScrewandWasher': 'text', 'ImpellerKey': 'text', 'Coating': 'text', 'BOM': 'text', 'PriceID': 'pointer', 'LeadtimeID': 'pointer'}


In [68]:
# Class to insert Instance w/ class, chartype
class Instance:
    def __init__(self, name, parent_element):
        self.name = name
        self.parent = parent_element
        
        self.attr = {"name": name}
        self.chartype_attr = {"value": "doublebyte"} # Sets chartype, which is always a doublebyte
        self.class_attr = {"name": self.parent} # Sets Class for each row/entry from PSD (Should be the same for all entries)

    def __str__(self):
        return  f'Instance XML Element with name: {self.name}'

    def create_element(self):
        xml_instance  = ET.SubElement(self.parent, "instance", attrib=self.attr)
        xml_class     = ET.SubElement(xml_instance, "class")
        xml_sub_class = ET.SubElement(xml_class, "ref", attrib=self.class_attr)           
        xml_chartype  = ET.SubElement(xml_instance, "chartype", attrib=self.chartype_attr)# Inserts chartype attribute

        as_string = ET.tostring(self.parent, encoding='us-ascii', method='xml')
        return self.parent

        def add_text_attribute(self, attributes_dict):
            attr_entry = ET.SubElement(xml_instance, "attribute", attributes_dict)

        def add_pointer_attribute(self, attributes_dict):
            attr_entry = ET.SubElement(xml_instance, "attribute", attributes_dict)
            pointer_sub_entry = ET.SubElement(attr_entry, "ref", pointer_dict)

In [ ]:
# Try to incorporate the Instance class 
root = ET.Element('top')  # Root element

# Iterate through dataframe in correct order to produce XML instance
for index, col in psd_data[start_row:end_row].iterrows():
    
    # Inserts Instance for each row in PSD
    psd_entry = Instance(col["ID"], root)
    new_xml_str = psd_entry.create_element()

    # Loop through header dictionary for attributes
    for k, v in header_dict.items():

        # Need to insert if statement where "pointers" receive "ref" sub-child
        if v == "text":
            attr_dict = {"name": k, "type": v, "value": psd_data.at[index, k].__str__()}    
            psd_entry.add_text_attribute(attr_dict)
        # Whats the difference between pointer and pointer-merge? May have to change this. 
        elif "pointer" in v: 
            attr_dict = {"name": k, "type": v}
            pointer_dict = {"value": psd_data.at[index, k].__str__()}
            psd_entry.add_pointer_attribute(attr_dict, pointer_dict)

In [73]:
# Iterate through dataframe in correct order to produce XML instance
root = ET.Element('top')  # Root element

# Sets Class for each row/entry from PSD (Should be the same for all entries)
class_ref_name = psd_data.columns[0] # This uses the PSD column header for the class name.
class_attr = {"name": class_ref_name}

# Sets chartype, which is always a doublebyte
chartype_attr = {"value": "doublebyte"}

for index, col in psd_data[start_row:end_row].iterrows():
    # Inserts Instance for each row in PSD
    inst_attr = {"name": col["ID"]}
    instance = ET.SubElement(root, "instance", attrib=inst_attr)

    # Inserts Class for each row in PSD
    class_entry = ET.SubElement(instance, "class")
    class_sub_entry = ET.SubElement(class_entry, "ref", attrib=class_attr)

    # Inserts chartype attribute
    chartype = ET.SubElement(instance, "chartype", chartype_attr)

    # Loop through header dictionary for attributes
    for k, v in header_dict.items():
        # Need to insert if statement where pointers receive sub-child
        if v == "text":
            attr_dict = {"name": k, "type": v, "value": psd_data.at[index, k].__str__()}
            attr_entry = ET.SubElement(instance, "attribute", attr_dict)
        elif "pointer" in v: # Whats the difference between pointer and pointer-merge? May have to change this. 
            attr_dict = {"name": k, "type": v}
            attr_entry = ET.SubElement(instance, "attribute", attr_dict)
            pointer_dict = {"value": psd_data.at[index, k].__str__()}
            pointer_sub_entry = ET.SubElement(attr_entry, "ref", attrib=pointer_dict)

In [ ]:
prettify(root)